# Find Lane Lines Testbed

In [ ]:
import cv2
import argparse
import numpy as np
import glob
import pickle
from moviepy.editor import VideoFileClip
from calibration import calculate_mtx_dist
from find_lines import *
import matplotlib.pyplot as plt
%matplotlib inline


video_file = 'project_video.mp4'
output_video_file = 'output.mp4'
mtx_dist_file = 'dist_pickle.p'
verbose = True


print("Video file: {}".format(video_file))
print("Output video file: {}".format(output_video_file))
print("Mtx/Dist file: {}".format(mtx_dist_file))
print("Verbose: {}".format(verbose))

print("Find lane lines ...")

# Load Saved Camera Matrix and Distortion Coefficients
dist_pickle = pickle.load(open(mtx_dist_file, "rb" ))
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

if verbose:
    print('mtx=',mtx)
    print('dist=',dist)


def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0,255)):
    
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'y':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)

    # print(sobel)
    
    # 3) Take the absolute value of the derivative or gradient
    sobel_abs = np.absolute(sobel)
    # print(sobel_abs)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    sobel_scaled = np.uint8(255*sobel_abs/np.max(sobel_abs))
    # print(sobel_scaled)
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    msk = np.zeros_like(sobel_scaled)
    msk[(sobel_scaled > thresh[0]) & (sobel_scaled < thresh[1]) ] = 1
    # print(msk)
    # 6) Return this mask as your binary_output image
    binary_output = np.copy(msk) # Remove this line
#     return binary_output
    return binary_output

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, sobel_kernel)
    
    # 3) Calculate the magnitude 
    sobel_magn = np.sqrt(sobelx * sobelx + sobely * sobely)
    
    # 5) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    sobel_scaled = np.uint8(255*sobel_magn/np.max(sobel_magn))
    
    # 6) Create a binary mask where mag thresholds are met
    msk = np.zeros_like(sobel_scaled)
    msk[(sobel_scaled > mag_thresh[0]) & (sobel_scaled < mag_thresh[1]) ] = 1
    
    # 7) Return this mask as your binary_output image
    binary_output = np.copy(msk) # Remove this line
    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    dir1 = np.arctan2(abs_sobely, abs_sobelx)
#     print(dir1)
    # 5) Create a binary mask where direction thresholds are met
    msk = np.zeros_like(dir1)
    msk[(dir1 >= thresh[0]) & (dir1 <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    binary_output = np.copy(msk) # Remove this line
    return binary_output

    
class LineFinder(object):
    def __init__(self, mtx, dist, samples=0.1):
        self.counter = 0
        self.mtx = mtx
        self.dist = dist
        self.samples_orig = []
        self.samples_proc = []
    def process_image(self, image):
        img = np.copy(image)
        
        sampled = True if np.random.uniform() < 0.1 else False
        if sampled:
            self.samples_orig.append(image)
        
        # Frame number
#         w, h = self.size
        w, h = image.shape[1], image.shape[0]

        ksize = 3
        thresh_abs = (50, 120)
        thresh_mag = (50, 120)
        # thresh_dir = (0.7, 1.3)
        thresh_dir = (0.75, 1.15)
        
        img = cv2.undistort(img, mtx, dist, None, mtx)
        
        gradx = abs_sobel_thresh(img, orient='x', sobel_kernel=ksize, thresh=(50, 120))
#         print('gradx=', np.mean(gradx))
        img = cv2.cvtColor(gradx*255, cv2.COLOR_GRAY2RGB)
#         print('imgg=', np.mean(imgg))
#         if sampled:
#             plt.figure()
#             plt.imshow(imgg)

        
        cv2.putText(img, "{}".format(self.counter), (int(w/2), 30), cv2.FONT_HERSHEY_SIMPLEX, 1, [255, 255, 255], 2)
        
        if sampled:
            self.samples_proc.append(img)
        
        self.counter += 1
#         print('self.counter =', self.counter)
        return img
    



clip = VideoFileClip(video_file)
clip = clip.subclip(t_end=20)
lineFinder = LineFinder(mtx=mtx, dist=dist, samples=0.01)
clip = clip.fl_image(lineFinder.process_image)
clip.write_videofile(output_video_file, audio=False)

# img = cv2.imread('test_images/solidYellowLeft.jpg')
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# lineFinder = LineFinder(mtx=mtx, dist=dist)
# lineFinder.process_image(img)

print('len(lineFinder) =', len(lineFinder.samples_orig))


for idx, (sample_orig, sample_proc) in enumerate(zip(lineFinder.samples_orig, lineFinder.samples_proc)):
#     print('idx=', idx)
    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    cur_axes = plt.gca()
    cur_axes.axes.get_xaxis().set_ticks([])
    cur_axes.axes.get_yaxis().set_ticks([])
    if idx == 0:
        plt.title('Original Image', fontsize=20)
    plt.imshow(sample_orig)
    plt.subplot(1, 2, 2)
    cur_axes = plt.gca()
    cur_axes.axes.get_xaxis().set_ticks([])
    cur_axes.axes.get_yaxis().set_ticks([])
    if idx == 0:
        plt.title('Processed', fontsize=20)
    plt.imshow(sample_proc)



Video file: project_video.mp4
Output video file: output.mp4
Mtx/Dist file: dist_pickle.p
Verbose: True
Find lane lines ...
mtx= [[  1.15396093e+03   0.00000000e+00   6.69705357e+02]
 [  0.00000000e+00   1.14802496e+03   3.85656234e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]
dist= [[ -2.41017956e-01  -5.30721173e-02  -1.15810355e-03  -1.28318856e-04
    2.67125290e-02]]
[MoviePy] >>>> Building video output.mp4
[MoviePy] Writing video output.mp4


 61%|██████    | 304/501 [00:24<00:15, 12.61it/s]

In [4]:

from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_video_file))

